In [1]:
def prot_mut(pdb_path, pdb_file, pqr_output_path, locked_aa_pos=None, Deep_mut=True, iterations=100, cutoff_value = -0.005, threshhold = 10000, seed = 0, remove_files = None):
    """
    This function performs protein mutation analysis to improve the thermal stability of a protein, with minimal changes to the structure
    (as measured by melting point). It takes a PDB file path, filename, and path for PQR output as input.

    Args:
        pdb_path (str): Path where the pdb file is stored.
        pdb_file (str): Name of the PDB file.
        pqr_output_path (str): Path where pqr file will be saved (also fasta and fas).
        locked_aa_pos (list, optional): List of amino acid positions that should not be mutated (defaults to None).
        Deep_mut (bool, optional): Flag whether to include random mutation in screening
            (random + rational, defaults to True).
        iterations (int, optional): Number of iterations for rational improvement (defaults to 100).
        cutoff_value (float, optional): Cutoff value for mutation selection in rational improvement 
            (defaults to -0.005).
        threshhold (int, optional): Threshold for random mutation acceptance (higher value leads to more mutations, 
            defaults to 10000).
        seed (int, optional): Seed for the random number generator (ensures reproducibility, defaults to 0).

    Returns:
        list: A list containing three elements:
            - Tuple: (WT_SPARC object, best_SPARC object) - Wild-type and best mutated protein SPARC predictions.
            - Tuple: (WT amino acid list, best mutated protein amino acid list) - Amino acid sequences.
            - Tuple: (WT deviation sum, best mutated protein deviation sum) - Deviations of the protein structures.
    """
    #import functions

    from ThERMOS import AA2s4pred
    from ThERMOS import diff_weighted
    from ThERMOS import mutator_rand
    from ThERMOS import mutator_rational
    from ThERMOS import functional_aa
    from ThERMOS import free_aa
    from ThERMOS import Subst_reducer
    from ThERMOS import pdb2AA
    from ThERMOS import ArraySlice
    from ThERMOS import pos_corr, neg_corr, ideal_pos_value, ideal_neg_value, conserv_subst, non_conservative_substitutions

    from SPARC import SPARC
    
    from heapq import heappop, heappush
    import heapq    
    import os
   

    #extract protein features
    aa_list = pdb2AA(pdb_path, pdb_file)
    aa_locked = functional_aa(pdb_path, pdb_file, pqr_output_path)
    aa_free = free_aa(pdb_path, pdb_file, aa_locked)
    aa_str = ''.join(aa_list)
    free_AA_dict = {a: b for a, b in zip(aa_free[:,2], aa_free[:,1] )} # create dictionary from array the key is the absolute aminoacid position and value is the aminoacid
    
    if locked_aa_pos is not None:
        for pos in locked_aa_pos:
            free_AA_dict.pop(pos)
    
    sec_prediction = AA2s4pred('./data/s4pred', pqr_output_path, aa_str, pdb_file, remove_file = remove_files)
    possible_substitutions = Subst_reducer(sec_prediction, conserv_subst, free_AA_dict, seed = seed)


    #calculate WT deviations
    WT_dev_sum, WT_dev = diff_weighted(pos_corr, neg_corr, aa_list, ideal_pos_value, ideal_neg_value, sec_prediction)
    """
     if Deep_mut:
        #randoly mutate protein (within given constraints), get top 10 mutations
        top_variations = []
        largest_variations = []
        heappush(top_variations, (WT_dev_sum, WT_dev, aa_str))  # Placeholder for lowest score
        heappush(largest_variations, (0, WT_dev, aa_str))  # Placeholder for largest variation
        
        Mut_seq_str = mutator_rand(aa_list, possible_substitutions, threshhold = threshhold, seed = seed)
        for Mut_prot in Mut_seq_str:
            Mut_dev_sum, Mut_dev = diff_weighted(pos_corr, neg_corr, Mut_prot, ideal_pos_value, ideal_neg_value, sec_prediction, sort = True)
            Str_dev = sum(c1 != c2 for c1, c2 in zip(''.join(Mut_prot), aa_str))

            #save the top 10 scores
            if len(top_variations) < 11:
                heappush(top_variations, (Mut_dev_sum, Mut_dev, Mut_prot)) 
                    
            elif Mut_dev_sum < top_variations[0][0]:
                heappush(top_variations, (Mut_dev_sum, Mut_dev, Mut_prot))
                heappop(top_variations)
            
            #saves top 10 largest variations
            if len(largest_variations) <11:
                heappush(largest_variations, (Str_dev, Mut_dev, Mut_prot))
            elif Str_dev > largest_variations[0][0]:
                heappush(largest_variations, (Str_dev, Mut_dev, Mut_prot))
                heappop(largest_variations)
                
        heappush(top_variations, (WT_dev_sum, WT_dev, aa_str))
        Random_creation = list(heapq.merge(top_variations, largest_variations))
        print('Random mutation finished')
        
        
    #define variables for iteration
    prev_Mut_prot_list = aa_list    
    prev_Mut_dev = WT_dev           
    aa_available = aa_free          
    
    #define variables for best protein#
    if Deep_mut:
        best_Mut_prot_list = list(heappop(top_variations)[2])
        best_Mut_dev_sum = heappop(top_variations)[0]
        best_Mut_dev = heappop(top_variations)[1]
        best_aa_available = aa_available
    else:
        best_Mut_prot_list = list(aa_list)
        best_Mut_dev_sum = WT_dev_sum
        best_Mut_dev = WT_dev
        best_aa_available = aa_available
        Random_creation = [(WT_dev_sum, WT_dev, aa_str)]
    
    # Initiate top 5 best variations of rational improvement to calculate melt point
    top_top_variations = []
    heappush(top_top_variations, (WT_dev_sum, WT_dev, aa_str))
    
    
    #use top 10 mutated sequences and use rational improvement      
    best_iteration = 0 #(used to track how many iterations are necessary, currently ~2-3 seems best)
    for mut_seq in Random_creation:
        prev_Mut_prot_list = list(mut_seq[2])
        prev_Mut_dev_sum = mut_seq[0]
        prev_Mut_dev = mut_seq[1]

        for k in range(iterations):
            Mut_prot_list, possible_mutations = mutator_rational(
                                                    AA_list = prev_Mut_prot_list, 
                                                    free_AA = aa_available, 
                                                    deviation = prev_Mut_dev,
                                                    pos_corr = pos_corr, 
                                                    neg_corr =  neg_corr, 
                                                    conserv_substitution = possible_substitutions,
                                                    ideal_pos_value = ideal_pos_value, 
                                                    ideal_neg_value = ideal_neg_value,
                                                    cutoff = cutoff_value,
                                                    sec_prediction = sec_prediction
                                                    ) #f_value = cutoff, calculates list of possible mutations
            
            Mut_dev_sum, Mut_dev = diff_weighted(pos_corr, neg_corr, Mut_prot_list, ideal_pos_value, ideal_neg_value, sec_prediction) # calculate deviation of mutated protein sequence
            
            if len(top_top_variations) < 6:
                heappush(top_top_variations, (Mut_dev_sum, Mut_dev, Mut_prot_list))
            elif Mut_dev_sum < top_top_variations[0][0]:
                heappush(top_top_variations, (Mut_dev_sum, Mut_dev, Mut_prot_list))
                heappop(top_top_variations)
                
            if abs(best_Mut_dev_sum) > abs(Mut_dev_sum):
                best_Mut_prot_list = Mut_prot_list
                best_Mut_dev_sum = Mut_dev_sum
                best_Mut_dev = Mut_dev  
                best_possible_mutations = possible_mutations #get list of best mutations (AA-POS-AA), depreciated, bcs random mutator doesn't output this
                best_iteration = str(k+1)
                #aa_available = ArraySlice(aa_available, possible_mutations) #updates available aminoacids, so that each aminoacid can only be mutated once
                
            elif Mut_dev[0][0] == prev_Mut_dev[0][0] and abs(Mut_dev[0][1]-prev_Mut_dev[0][1]) < 0.001:
                break
                
            #update variables for next iteration            
            prev_possible_mutations = possible_mutations    #list of mutations (AA-POS-AA) (prev_possible_mutations can be printed if needed)
            prev_Mut_prot_list = Mut_prot_list                        #Mutated protein as a list with one AA per entry
            prev_Mut_dev = Mut_dev
            prev_Mut_dev_sum = Mut_dev_sum
    
        
    #for top_hit in top_top_variations:
    #-------SPARC implementation missing--------#
    wt_sparc = SPARC(aa_str, pdb_file.split('-')[1], './data', './data/s4pred')
    best_temp = wt_sparc[0]
    wt_temp = wt_sparc[0]  
    best_SPARC = wt_sparc
    
    #select best mutation based on melt point
    for top in top_top_variations:
        top_SPARC = SPARC(''.join(top[2]), pdb_file.split('-')[1], './data', './data/s4pred')
        top_temp = top_SPARC[0]
        if top_SPARC[0] > best_temp:
            best_temp = top_SPARC[0]
            best_SPARC = top_SPARC
            best_Mut_prot_list = top[2]
            best_Mut_dev_sum = top[0]
            best_Mut_dev = top[1] """
    
    
    
    #Improvement = WT_dev_sum - best_Mut_dev_sum
    
    return aa_list, possible_substitutions

In [2]:
aa_list, poss_subst = prot_mut('./data/pdbs', 'AF-C0H3V2-F1.pdb', './data/test' )

Pqr file already exists
fasta file already exists
fas file already exists


In [1]:

def prot_mut_worker(args):
    from ThERMOS import prot_mut
    pdb_file_path, pqr_output_path, locked_aa_pos, Deep_mut, iterations, cutoff_value, threshhold, seed, remove_files = args
    return prot_mut(pdb_file_path, pqr_output_path, locked_aa_pos, Deep_mut, iterations, cutoff_value, threshhold, seed, remove_files)

def run_prot_mut_for_multiple_proteins_multiprocessing(pdb_file_paths, pqr_output_path, locked_aa_pos=None, Deep_mut=True, iterations=100, cutoff_value = -0.005, threshhold = 10000, seed = 0, remove_files = None, num_processes=4):
    from multiprocessing import Pool
    with Pool(processes=num_processes) as pool:
        results = pool.starmap(prot_mut_worker, [(pdb_file_path, pqr_output_path, locked_aa_pos, Deep_mut, iterations, cutoff_value, threshhold, seed, remove_files) for pdb_file_path in pdb_file_paths])
    return results



## Mutational screening of functional proteins


#### **Mutational screen of all esssential proteins**

In [17]:
from ThERMOS import prot_mut
test = prot_mut('./data/pdbs', 'AF-C0H3V2-F1.pdb', './data/test', iterations = 1, threshhold = 1)

Pqr file already exists
fasta file already exists
fas file already exists
Random mutation finished


In [18]:
print(test[0][0][0][0])
print(test[0][1][0][0])
print(test[1][0])
print(test[1][1])

55.84154306316571
55.84154306316571
['M', 'Q', 'V', 'L', 'A', 'K', 'E', 'N', 'I', 'K', 'L', 'N', 'Q', 'T', 'V', 'S', 'S', 'K', 'E', 'E', 'A', 'I', 'K', 'L', 'A', 'G', 'Q', 'T', 'L', 'I', 'D', 'N', 'G', 'Y', 'V', 'T', 'E', 'D', 'Y', 'I', 'S', 'K', 'M', 'F', 'E', 'R', 'E', 'E', 'T', 'S', 'S', 'T', 'F', 'M', 'G', 'N', 'F', 'I', 'A', 'I', 'P', 'H', 'G', 'T', 'E', 'E', 'A', 'K', 'S', 'E', 'V', 'L', 'H', 'S', 'G', 'I', 'S', 'I', 'I', 'Q', 'I', 'P', 'E', 'G', 'V', 'E', 'Y', 'G', 'E', 'G', 'N', 'T', 'A', 'K', 'V', 'V', 'F', 'G', 'I', 'A', 'G', 'K', 'N', 'N', 'E', 'H', 'L', 'D', 'I', 'L', 'S', 'N', 'I', 'A', 'I', 'I', 'C', 'S', 'E', 'E', 'E', 'N', 'I', 'E', 'R', 'L', 'I', 'S', 'A', 'K', 'S', 'E', 'E', 'D', 'L', 'I', 'A', 'I', 'F', 'N', 'E', 'V', 'N']
['M', 'Q', 'V', 'L', 'A', 'K', 'E', 'N', 'I', 'K', 'L', 'N', 'Q', 'T', 'V', 'S', 'S', 'K', 'E', 'E', 'A', 'I', 'K', 'L', 'A', 'G', 'Q', 'T', 'L', 'I', 'D', 'N', 'G', 'Y', 'V', 'T', 'E', 'D', 'Y', 'I', 'S', 'K', 'M', 'F', 'E', 'R', 'E', 'E', 'T', 'S

In [1]:
import pandas as pd
import os
import pickle
from ThERMOS import prot_mut
data_path = './data/essential_proteins_final.csv'
ess_prot_AFdb = (
    pd.read_csv(data_path)['AlphaFoldDB']
    .dropna()  
    .str.replace(';', '')  
    .apply(lambda x: f'AF-{x}-F1.pdb') 
)
ess_prot_AFdb = ess_prot_AFdb.tolist()
path_pdb = './data/pdbs'
path_pqr=  './data/pqrs'
output_path = './data/test'

Mutator_dict = {}
count = 6
no_pdb = []
for entry in ess_prot_AFdb[5:]:
    if os.path.exists(os.path.join(path_pdb, entry)):
        Mutator_dict[entry] = prot_mut(path_pdb, entry, path_pqr, Deep_mut=True, iterations=100, cutoff_value = -0.005, threshhold = 10000, seed = 0)
    else:
        no_pdb.append(entry)
        print(f'{entry} not found')
    with(open('./data/Mutator_rational_final.pkl', 'wb')) as file:
        pickle.dump(Mutator_dict, file)
    print(f'{entry} finished ({count}/{len(ess_prot_AFdb)})')
    count += 1



Pqr file already exists
fasta file already exists
fas file already exists


KeyboardInterrupt: 

In [9]:
test = [1,2,3,4,5,6,7,8]
print(test[5:])

[6, 7, 8]


In [11]:
import pickle

with(open('./data/Mutator_rational_02.pkl', 'rb')) as file:
    test = pickle.load(file)

print(test['AF-O07615-F1.pdb'][0][1][0])

[86.22309223]


#### **Analyze mutation sreen of essential proteins**

In [ ]:
import pickle
with(open('./data/Essential_genes_mutated.pkl', 'rb')) as f:
    Mutator_dict_load = pickle.load(f)
with(open('./data/Mutator_dict_13first_bs_error.pkl', 'rb')) as f:
    Mutator_dict_13 = pickle.load(f)
Mutated_proteins = {**Mutator_dict_load, **Mutator_dict_13}

""" with(open('./data/Mutator_rational_test_top79.pkl', 'rb')) as file:
    Mutated_proteins = pickle.load(file) """


Increase = 0
No_improvement = []
Mutated_improvement = {}
for key in Mutated_proteins:
    if Mutated_proteins[key][0][0][0] == Mutated_proteins[key][0][1][0]:
        No_improvement.append(key)
    else:
        Mutated_improvement[key] = Mutated_proteins[key]

print(Mutated_improvement.keys())

Diff = 0
Changed_aa_amount = 0
highest_increase = ('name',0)
biggest_relative_change = ('name',0)
biggest_absolute_chagne = ('name',0)
rel_mut_increase = ('name',0)
for key in Mutated_improvement:
    #Melt point increase
    Diff += float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0])
    
    #highest increase
    if float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0]) > highest_increase[1]:
        highest_increase = (key, float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0]))
    
    #percentage of changed aminoacids
    changed_aa = 0
    for n in range(len(Mutated_improvement[key][1][0])): 
        if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
            changed_aa += 1
    if changed_aa/len(Mutated_improvement[key][1][0]) > biggest_relative_change[1]:
        biggest_relative_change = (key, changed_aa/len(Mutated_improvement[key][1][0]))
    if changed_aa > biggest_absolute_chagne[1]:
        biggest_absolute_chagne = (key, changed_aa)
    Changed_aa_amount += changed_aa/len(Mutated_improvement[key][1][0])
    if changed_aa > 0:
        if (float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0]))/float(changed_aa) > rel_mut_increase[1]:
            rel_mut_increase = (key, (float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0]))/float(changed_aa))

#Mutation list for edge cases
highest_increase_mut_list = []
biggest_rel_change_mut_list = []
biggest_abs_change_mut_list =[]
highest_mut2melt_list = []
for key in Mutated_improvement:
    if key == highest_increase[0]:
        for n in range(len(Mutated_improvement[key][1][0])):
            if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
                highest_increase_mut_list.append(f'{n+1}')
    if key == biggest_relative_change[0]:
        for n in range(len(Mutated_improvement[key][1][0])):
            if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
                biggest_rel_change_mut_list.append(f'{n+1}')
    if key == biggest_absolute_chagne[0]:
        for n in range(len(Mutated_improvement[key][1][0])):
            if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
                biggest_abs_change_mut_list.append(f'{n+1}')
    if key == rel_mut_increase[0]:
        for n in range(len(Mutated_improvement[key][1][0])):
            if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
                highest_mut2melt_list.append(f'{n+1}')


#Print results
#print(f'Proteins without a pdb file: {len(no_pdb)}')
print(f"Proteins that couldn't be improved: {len(No_improvement)}")
print(f'Proteins that could be improved: {len(Mutated_improvement.keys())} \n')
print(f'Average meltPoint increase: {Diff/len(Mutated_improvement.keys())}')
print(f'Average percentage of changed aminoacid per protein: {Changed_aa_amount/len(Mutated_improvement.keys())} \n')
print(f'Highest meltPoint increase: {highest_increase[0]} with {highest_increase[1]} ({Mutated_improvement[highest_increase[0]][0][0][0][0]} -> {Mutated_improvement[highest_increase[0]][0][1][0][0]})\nWT sequence:      {''.join(Mutated_improvement['AF-P37948-F1.pdb'][1][0])} \nMutated sequence: {''.join(Mutated_improvement['AF-P37948-F1.pdb'][1][1])}')
print(f'{'+'.join(highest_increase_mut_list)}\n')
print(f'Highest percentage of changed aminoacid: {biggest_relative_change[0]} with {biggest_relative_change[1]} with {int(len(Mutated_improvement[biggest_relative_change[0]][1][0])*biggest_relative_change[1])} of {len(Mutated_improvement[biggest_relative_change[0]][1][0])}\nWT sequence:      {''.join(Mutated_improvement['AF-Q47710-F1.pdb'][1][0])} \nMutated sequence: {''.join(Mutated_improvement['AF-Q47710-F1.pdb'][1][1])}')
print(f'{'+'.join(biggest_rel_change_mut_list)}\n')
print(f'Highest absolute number of changed aminoacid: {biggest_absolute_chagne[0]} with {biggest_absolute_chagne[1]} of {len(Mutated_improvement[biggest_absolute_chagne[0]][1][0])}\nWT sequence:      {''.join(Mutated_improvement['AF-P46918-F1.pdb'][1][0])} \nMutated sequence: {''.join(Mutated_improvement['AF-P46918-F1.pdb'][1][1])}')
print(f'{'+'.join(biggest_abs_change_mut_list)}\n')                                                                                                                                                                                                                                         
print(f'')
print(f'Biggest meltPoint change per mutation {rel_mut_increase[0]} with {rel_mut_increase[1]} ({Mutated_improvement[rel_mut_increase[0]][0][0][0][0]} -> {Mutated_improvement[rel_mut_increase[0]][0][1][0][0]})\nWT sequence:      {''.join(Mutated_improvement[rel_mut_increase[0]][1][0])} \nMutated sequence: {''.join(Mutated_improvement[rel_mut_increase[0]][1][1])}')
print(f'{'+'.join(highest_mut2melt_list)}\n')


#### **Create dataframe from mutation results for statistical testing and plotting**

In [ ]:
import pickle
import pandas as pd
with(open('./data/Essential_genes_mutated.pkl', 'rb')) as f:
    Mutator_dict_load = pickle.load(f)
with(open('./data/Mutator_dict_13first_bs_error.pkl', 'rb')) as f:
    Mutator_dict_13 = pickle.load(f)
Mutated_proteins = {**Mutator_dict_load, **Mutator_dict_13}
Mutated_df = pd.DataFrame(columns = ['Protein', 'WT_prediction', 'Mut_prediction', 'WT_sequence', 'Mut_sequence', 'Mutations'])
for key in Mutated_proteins:
    wt_prediction = Mutated_proteins[key][0][0][0][0]
    mut_prediction = Mutated_proteins[key][0][1][0][0]
    wt_seq = ''.join(Mutated_proteins[key][1][0])
    mut_seq = ''.join(Mutated_proteins[key][1][1])
    mutations_list = []
    for i in range(len(wt_seq)):
        if wt_seq[i] != mut_seq[i]:
            mutations_list.append(f'{wt_seq[i]}{i+1}{mut_seq[i]}')
    new_row = [key.split('-')[1], wt_prediction, mut_prediction, wt_seq, mut_seq, mutations_list]
    #print(new_row)
    Mutated_df.loc[len(Mutated_df)] = new_row

print(Mutated_df.loc[1])

with(open('./data/Mutated_proteins_df.pkl', 'wb')) as f:
    pickle.dump(Mutated_df, f)